In [1]:
from pyspark.sql import SparkSession
from scipy import sparse
import numpy as np
import pandas as pd
from pyspark.ml.recommendation import ALS
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from  pyspark.sql.functions import isnan
# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()

# Convert a Pandas DF to a Spark DF
#3spark_df = spark.createDataFrame(pandas_df) 

# Convert a Spark DF to a Pandas DF
#pandas_df = spark_df.toPandas()

In [2]:
df = pd.read_csv('../data/training.csv')
user_df = pd.read_csv('../data/users.dat', sep = '::', header = None)

movies_data = pd.read_csv('../data/movies.dat', sep = '::', header = None)
movies_data.columns = ['movie','name','genre']



/Users/Haebichan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/Users/Haebichan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  after removing the cwd from sys.path.


In [3]:
user_df.columns = ['user', 'gender', 'age', 'occupation', 'zipcode']
df_merged = df.set_index('user').join(user_df.set_index('user'))
df_merged.reset_index(inplace = True)

In [4]:

movie_merged = movies_data.set_index('movie').join(df.set_index('movie'))
movie_merged.reset_index(inplace = True)

In [306]:
# final_merge = movie_merged.set_index('user').join(df_merged.set_index('user'))

In [307]:
# final_merge.reset_index(inplace = True)

In [22]:
def get_genders(df_merged, df):
    female_user = df_merged[df_merged['gender'] == 'F']
    male_user = df_merged[df_merged['gender'] == 'M']
    male_user_list = list(male_user.movie)
    female_user_list = list(female_user.movie)
    male_train = df[df['user'].isin (male_user_list)]
    female_train = df[df['user'].isin (female_user_list)]
    return male_train, female_train

In [30]:
female_user = df_merged[df_merged['gender'] == 'F']
male_user = df_merged[df_merged['gender'] == 'M']

In [31]:
female_user.drop('gender',1, inplace = True)
male_user.drop('gender',1, inplace = True)

/Users/Haebichan/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
female_user = df_merged[df_merged['gender'] == 'F']
male_user = df_merged[df_merged['gender'] == 'M']

In [320]:
male_user_list = list(male_user.movie)

In [321]:
female_user_list = list(female_user.movie)

In [322]:
male_train = df[df['user'].isin (male_user_list)]

In [323]:
female_train = df[df['user'].isin (female_user_list)]

In [349]:
def get_genders(user_df, df):
    female_user = df_merged[df_merged['gender'] == 'F']
    male_user = df_merged[df_merged['gender'] == 'M']
    male_user_list = list(male_user.movie)
    female_user_list = list(female_user.movie)
    male_train = df[df['user'].isin (male_user_list)]
    female_train = df[df['user'].isin (female_user_list)]
    return male_train, female_train

In [327]:
change

<function __main__.change(x, other_df)>

In [328]:
df.user.describe()

count    800000.000000
mean       3403.097837
std        1546.589028
min         636.000000
25%        2035.000000
50%        3507.000000
75%        4695.000000
max        6040.000000
Name: user, dtype: float64

In [329]:
df.isnull().sum()

user         0
movie        0
rating       0
timestamp    0
dtype: int64

In [330]:
df.shape

(800000, 4)

In [1]:
male_ratings_df = spark.createDataFrame(female_user)
female_ratings_df = spark.createDataFrame(female_train)

In [ ]:
als = ALS(
    itemCol='movie',
    userCol='user',
    ratingCol='rating',
    nonnegative=True,    
    regParam=0.1,
    rank=10)

In [32]:
male_recommender = als.fit(male_user)
female_recommender = als.fit(female_user)

NameError: name 'als' is not defined

In [ ]:
male_train_recs = male_recommender.transform(male_ratings_df)
female_train_recs = female_recommender.transform(female_ratings_df)

In [ ]:
male_pandas_df = male_train_recs.toPandas()
female_pandas_df = female_train_recs.toPandas()

In [336]:
male_pandas_df.describe()

,user,movie,rating,timestamp,prediction
count,441400.000000,441400.000000,441400.000000,4.414000e+05,441400.000000
mean,2372.701332,1862.145351,3.584304,9.722364e+08,3.432790
std,972.865509,1097.128733,1.122885,3.438265e+06,0.683172
min,637.000000,1.000000,1.000000,9.656765e+08,0.193766
25%,1541.000000,1029.000000,3.000000,9.686634e+08,3.026272
50%,2371.000000,1829.000000,4.000000,9.744350e+08,3.508238
75%,3265.000000,2762.000000,4.000000,9.747909e+08,3.919509
max,3952.000000,3952.000000,5.000000,9.757673e+08,5.444750


In [337]:
# pandas_df.rating.hist(bins=5,density=True)
# #plt.show()

In [338]:
# pandas_df.prediction.hist(bins=5,density=True)

In [339]:
test = pd.read_csv('../data/requests.csv')

In [351]:
test_male, test_female = get_genders(user_df, test)

In [352]:
male_test_ratings_df = spark.createDataFrame(test_male)
female_test_ratings_df = spark.createDataFrame(test_female)

In [ ]:
male_test_recs = male_recommender.transform(male_test_ratings_df)
female_test_recs = female_recommender.transform(female_test_ratings_df)

In [ ]:
male_test_recs = male_test_recs.toPandas()
female_test_recs = female_test_recs.toPandas()

In [ ]:
male_test_recs.rename(columns={'prediction': 'rating'}, inplace=True)
female_test_recs.rename(columns={'prediction': 'rating'}, inplace=True)
male_pandas_df.rename(columns={'prediction': 'rating'}, inplace=True)
female_pandas_df.rename(columns={'prediction': 'rating'}, inplace=True)

In [ ]:
male_pandas_df.head()

In [ ]:
male_test_recs.head()

In [ ]:
male_test_recs.fillna(male_test_recs['rating'].mean(), inplace=True)
female_test_recs.fillna(female_test_recs['rating'].mean(), inplace=True)

In [223]:
# result = male_test_recs.append(female_test_recs)

In [2]:
# result.head()

In [203]:
submission.to_csv('../data/mvp_submission.csv',index=False)

200209

In [227]:
result.to_csv('../data/haebichanrocks.csv')

In [188]:
# pandas_df.head()

In [15]:
test = pd.read_csv('../data/requests.csv')

In [16]:
test.shape

(200209, 2)

In [17]:
test_ratings_df = spark.createDataFrame(test)

In [18]:
test_recs = recommender.transform(test_ratings_df)

In [19]:
test_recs.select(isnan('prediction')).groupby('isnan(prediction)').count().show()

+-----------------+------+
|isnan(prediction)| count|
+-----------------+------+
|             true| 95770|
|            false|104439|
+-----------------+------+



In [20]:
submission = test_recs.toPandas()

In [89]:
submission.rename(columns={'prediction': 'rating'}, inplace=True)

In [90]:
submission.to_csv('../data/mvp_submission.csv',index=False)

In [81]:
submission.head()

,user,movie,ratings
0,53,148,NaN
1,4169,148,3.117337
2,5333,148,2.508182
3,4387,148,1.868285
4,840,148,2.642798


In [219]:
def change(x, other_df):

    if pd.isna(x['rating']):
        overall_means = other_df['rating'].mean()
        userid = x['user']
        movieid = x['movie']
        user_means = other_df[other_df['user'] == userid]['rating'].mean()
        movie_means = other_df[other_df['movie'] == movieid]['rating'].mean()
        user_bias =  user_means - overall_means
        movie_bias = movie_means - overall_means
        
        if pd.isna(user_bias):
            user_bias = 0
        if pd.isna(movie_bias):
            movie_bias = 0
        x['rating'] = overall_means + user_bias + movie_bias
    return x

In [ ]:
def merge_df(x, other_df):

    if pd.isna(x['rating']):
        overall_means = other_df['rating'].mean()
        userid = x['user']
        movieid = x['movie']
        user_means = other_df[other_df['user'] == userid]['rating'].mean()
        movie_means = other_df[other_df['movie'] == movieid]['rating'].mean()
        user_bias =  user_means - overall_means
        movie_bias = movie_means - overall_means
        
        if pd.isna(user_bias):
            user_bias = 0
        if pd.isna(movie_bias):
            movie_bias = 0
        x['rating'] = overall_means + user_bias + movie_bias
    return x

In [46]:
submission = submission.apply(change, args=(self.train,), axis=1)

In [21]:
df_movies = pd.read_csv('../data/movies.dat', sep = '::', header = None, skiprows = 1) 
df_movies = pd.read_csv('../data/movies.dat', sep = '::', header = None, skiprows = 1) 

/Users/cosmakufa/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,0,1,2
0,2,Jumanji (1995),Adventure|Children's|Fantasy
